In [1]:
from keras.applications import ResNet152
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
from xgboost import XGBRegressor

import progressbar
import h5py
import numpy as np

## Generate Labels

In [40]:
f_train = open("lamem/splits/train_1.txt").readlines()
f_test = open("lamem/splits/test_1.txt").readlines()
f_val = open("lamem/splits/val_1.txt").readlines()

X_train_list = [x[:12] for x in f_train]
Y_train = np.array([float(x[13:21]) for x in f_train])

X_test_list = [x[:12] for x in f_test]
Y_test = np.array([float(x[13:21]) for x in f_test])

X_val_list = [x[:12] for x in f_val]
Y_val = np.array([float(x[13:21]) for x in f_val])

In [46]:
with h5py.File("Y.h5", "w") as h5f:
    for i in range(len(Y_train)):
        h5f.create_dataset(X_train_list[i], data=Y_train[i])
    for i in range(len(Y_test)):
        h5f.create_dataset(X_test_list[i], data=Y_test[i])
    for i in range(len(Y_val)):
        h5f.create_dataset(X_val_list[i], data=Y_val[i])

In [32]:
h5f = h5py.File("test_features.h5")

X_test_valide = [x in set(h5f.keys()) for x in X_test_list]
X_train_list = [x for x, y in zip(X_test_list, X_test_valide) if y]
Y_train = [x for x, y in zip(Y_test, X_test_valide) if y]

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


## Load dataset

In [2]:
# with h5py.File("train_features.h5") as h5f:
#     X_train = np.array([h5f[img][:] for img in X_train_list])
h5f_train = h5py.File("train_features.h5")
h5f_test = h5py.File("test_features.h5")
X  = np.array([h5f_train[img][:] for img in list(h5f_train.keys())] + [h5f_test[img][:] for img in list(h5f_test.keys())])
h5f2 = h5py.File("Y.h5")
Y  = np.array([h5f2[img].value for img in list(h5f_train.keys())+list(h5f_test.keys())])

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  after removing the cwd from sys.path.
/Users/jojo/opt/an

In [3]:
X.shape

(52757, 2048)

In [4]:
train_set, test_set = train_test_split(range(len(X)), test_size=0.1)

## Prediction

### XGBoost regressor

In [33]:
def my_custom_loss_func(y_true, y_pred):
    c, p = spearmanr(y_true, y_pred)
    return c
score = make_scorer(my_custom_loss_func, greater_is_better=True)

In [34]:
ALGORITHMS = {
        'model': XGBRegressor(n_jobs=-1),
        'params': {
            "n_estimators": [100, 150, 200], # 'poly', 'sigmoid'
            "max_depth": [4, 5, 6],
            "eta": [0.01],
        }
    }
CV = 3
model = GridSearchCV(
        ALGORITHMS['model'],
        ALGORITHMS['params'],
        cv=CV,
        scoring=score,
        n_jobs=-1,
        )

In [35]:
model.fit(X[:7000], Y[:7000])

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             iid='warn', n_jobs=-

In [39]:
model = XGBRegressor(n_jobs=-1, n_estimators=200, max_depth=8, eta=0.01)
model.fit(X[:7000], Y[:7000])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.00999999978, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [40]:
pred = model.predict(X[7000:])
c, p = spearmanr(Y[7000:], pred)

print("Spearmanr Result: correlation: {:.5f}, p-value: {}".format(c, p))

Spearmanr Result: correlation: 0.59634, p-value: 4.339106950756918e-74


### SVM

In [196]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
ALGORITHMS = {
        'model': SVR(gamma='scale'),
        'params': {
            "kernel": ['rbf', 'sigmoid'], # 'poly', 'sigmoid'
            "C": [0.1, 1.0, 10.0],
            "epsilon": [0.001, 1.0],
        }
    }

CV = 3
model = GridSearchCV(
        ALGORITHMS['model'],
        ALGORITHMS['params'],
        cv=CV,
        scoring=score,
        n_jobs=-1,
        )

In [43]:
model = SVR(gamma='scale', kernel="rbf", C=1.0, epsilon=0.1)

In [ ]:
model.fit(X[train_set], Y[train_set])

In [ ]:
pred = model.predict(X[test_set])
c, p = spearmanr(Y[test_set], pred)

print("Spearmanr Result: correlation: {:.5f}, p-value: {}".format(c, p))

In [31]:
with h5py.File("test_features_new.h5", 'w') as h5f2:
    for i in set(h5f.keys()):
        try:
            h5f2.create_dataset(i, data=h5f[i].value)
        except:
            pass

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  after removing the cwd from sys.path.


### Neural network

In [5]:
from keras.layers import *
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras.backend import expand_dims
from keras.losses import MeanSquaredLogarithmicError
import tensorflow as tf
from tensorflow.keras.losses import MSE

def correlation_loss(x, y):    
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(y)
    xm, ym = x-mx, y-my
    r_num = tf.math.reduce_mean(tf.multiply(xm,ym))        
    r_den = tf.math.reduce_std(xm) * tf.math.reduce_std(ym)
    return 5 * MSE(x,y) - r_num / r_den

In [6]:
Y_new = (Y-np.mean(Y))/np.std(Y)

In [39]:
input_feature = Input(shape=(2048,), name='Input')
x = Dense(256, activation='sigmoid')(input_feature)
x = Dropout(0.5)(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

In [40]:
model = Model(input_feature, x)
model.compile(optimizer='adam', loss=correlation_loss)

In [41]:
model.fit(X[train_set], Y_new[train_set], batch_size=256, epochs=20, validation_data=(X[test_set], Y_new[test_set]))

Epoch 1/20
186/186 [==============================] - 1s 6ms/step - loss: 3.8278 - val_loss: 3.6755
Epoch 2/20
186/186 [==============================] - 1s 5ms/step - loss: 3.5339 - val_loss: 3.6340
Epoch 3/20
186/186 [==============================] - 1s 5ms/step - loss: 3.4715 - val_loss: 3.6092
Epoch 4/20
186/186 [==============================] - 1s 5ms/step - loss: 3.4313 - val_loss: 3.5816
Epoch 5/20
186/186 [==============================] - 1s 5ms/step - loss: 3.3912 - val_loss: 3.5776
Epoch 6/20
186/186 [==============================] - 1s 5ms/step - loss: 3.3646 - val_loss: 3.5686
Epoch 7/20
186/186 [==============================] - 1s 5ms/step - loss: 3.3329 - val_loss: 3.5700
Epoch 8/20
186/186 [==============================] - 1s 6ms/step - loss: 3.3072 - val_loss: 3.5827
Epoch 9/20
186/186 [==============================] - 1s 6ms/step - loss: 3.2774 - val_loss: 3.5800
Epoch 10/20
186/186 [==============================] - 1s 6ms/step - loss: 3.2581 - val_loss: 3.5853

In [42]:
pred = model.predict(X[test_set])
c, p = spearmanr(Y_new[test_set], pred)

print("Spearmanr Result: correlation: {:.5f}, p-value: {}".format(c, p))

Spearmanr Result: correlation: 0.66295, p-value: 0.0


In [184]:
model = ResNet152(weights="imagenet", include_top=False, pooling='avg')

In [185]:
IMG_SIZE=(224, 224)
image = load_img("frame/dyson1.mp4-frame-200.jpg", target_size=IMG_SIZE)
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [186]:
features = model.predict(image)

In [187]:
features.shape

(1, 2048)

In [195]:
model.predict(features)

array([[0.78836733]], dtype=float32)